# Maps and orbital geometry
It can be confusing to think about a map of a planet on an eccentric, inclined orbit at an arbitrary angle on the sky. Let's go through some examples that illustrate how `starry` handles 3D geometry in planet orbits. Note that this tutorial applies only to maps of `starry.Planet` instances. For maps instantiated straight up with `starry.Map`, there's no shenanigans: what you see is what you get!

## A simple example
Let's illustrate how things can get tricky in 3D with a simple example. To begin, let's say we have a planet on a circular, edge-on orbit with a simple dipole map:
```python
planet.map[1, 0] = 0.5
```
corresponding to a surface map that looks like this:

In [ ]:
import starry
import numpy as np
import matplotlib.pyplot as pl
%matplotlib inline
map = starry.Map()
map[1, 0] = 0.5
map.show()

Let's suppose further that the planet is rotating with $P_{rot} = P_{orb}$ and zero obliquity (i.e., what you'd expect for a hotspot that tracks the position of the star). In this case, the axis of rotation is the vector $\hat{y} = (0, 1, 0)$. Here's what the planet looks like on the sky over the course of one orbit:

In [ ]:
%run orbit_edgeon.py

No surprises here. During transit, the observer sees the full night side of the planet. During secondary eclipse, the observer sees the full day side. And at quadrature, the planet is half-illuminated.

**But here's where things could get tricky.** 

Suppose now that you have the same planet on an orbit that is inclined relative to the observer at $i = 60^\circ$. Suppose further that the line of nodes -- where the orbital plane intersects the reference plane (the sky plane in our case) -- is also misaligned with the $x$ axis at $\Omega = 30^\circ$. If we were to keep the exact same map -- the same map coefficients, and the same rotational axis relative to the sky plane -- this is what we would see:

In [ ]:
%run orbit_bad.py

If you look closely, this figure doesn't make any sense! The planet's dipole is no longer facing the star over the course of the orbit, and the rotational axis is no longer perpendicular to the orbital plane. That's because we kept the same map and the same rotational axis relative to the **sky plane**. But that's silly -- the planet's properties shouldn't depend on the vantage point of the observer. Instead, when we tell `starry` that the planet's map is
```python
planet[1, 0] = 0.5
```
and that the axis of rotation is
```
planet.axis = [0, 1, 0]
```
what we probably mean is that this is what the planet looks like **in the orbital plane.** Specifically, we want the planet to look like this **when the full dayside is visible**. So this is what we expect to see:

In [ ]:
%run orbit_good.py

That's **much** better. You can see that the hotspot always faces the star, and the axis of rotation is perpendicular to the orbital plane.

**Since it's easier to think about maps and rotations in the plane of the orbit, this second example is how** `starry` **actually works.** When you specify a map, that's what the planet looks like in its orbital plane, as if you were viewing the orbit edge-on, with the planet's full dayside visible (i.e., during secondary eclipse, if the star weren't blocking your view). When you specify an axis of rotation, that axis is always relative to the planet's orbital plane, meaning $\hat{y}$ is the vector perpendicular to the orbit, pointing "up". That's the default axis if none is provided.

## More examples
Let's look at some actual light curves now to see how `starry` behaves for different orbital orientations. To begin, let's instantiate a generic one-planet system with a very close-in, bright, tidally-locked planet:

In [ ]:
star = starry.kepler.Primary()
star[1] = 0.4
star[2] = 0.26
planet = starry.kepler.Secondary()
planet.L = 5e-3
planet.prot = 1
planet.porb = 1
planet.a = 5
system = starry.kepler.System(star, planet)
time = np.linspace(-0.1, 1.1, 100000)

## Edge-on circular orbit, planet with dipole map and centered hotspot

First off, the simplest possible case: an edge-on, circular orbit of a planet with a dipole map whose hotspot always points toward the star. Let's compute and plot the light curve over one orbit:

In [ ]:
planet.inc = 90
planet.ecc = 0
planet[1, 0] = 0.5
system.compute(time)
fig, ax = pl.subplots(1, figsize=(16, 4))
ax.plot(time, system.lightcurve);
ax.set_xlabel('Time [days]', fontsize=16);
ax.set_ylabel('System flux', fontsize=16);

This is exactly what we might expect: two transits at $t = 0$ and $t = 1$, a secondary eclipse at $t = 0.5$, and a phase curve that peaks during secondary eclipse. But it's worth being explicit about what's actually going on. As we mentioned above, when we instantiate a `Planet` map, we are telling `starry` that **this is what the planet looks like during secondary eclipse.** In this specific case, we told `starry` that our map looks like this:

In [ ]:
planet.show()

Since the map rotates around with the planet, the distinction we made above is necessary. **The coefficients of the map correspond to those of the map seen during secondary eclipse**, when the full dayside of the planet is (or would be, if the star weren't in the way) visible to the observer.

## Eccentric orbits

We can verify that this is still the case regardless of the eccentricity vector of the orbit:

In [ ]:
planet.inc = 90
planet.ecc = 0.5
planet.w = 30
planet[1, 0] = 0.5
system.compute(time)
fig, ax = pl.subplots(1, figsize=(16, 4))
ax.plot(time, system.lightcurve);
ax.set_xlabel('Time [days]', fontsize=16);
ax.set_ylabel('System flux', fontsize=16);

Secondary eclipse no longer occurs at the same time or mean longitude, but `starry` handles all the necessary rotations behind the scenes. When you specify a map, that is what you'll get when the planet is behind the star!

## Inclined orbits

Now what about inclined orbits, for which there may be no secondary eclipse? **Same thing**: we still specify the map as an observer would see it when looking at the system edge-on.

In [ ]:
fig, ax = pl.subplots(1, figsize=(16, 4))
planet.ecc = 0
planet[1, 0] = 0.5
inclinations = [0, 10, 30, 45, 60, 90]
for inc in inclinations:
    planet.inc = inc
    system.compute(time)
    ax.plot(time, planet.lightcurve, label=inc);
ax.set_xlabel('Time [days]', fontsize=16);
ax.set_ylabel('System flux', fontsize=16);
ax.legend(title='inclination (deg)', ncol=2);

This time I only plotted the *planet's* flux (to get rid of the transits and zoom in on the phase curves). As expected, as the inclination is reduced and the orbit becomes ever more face-on, the amplitude of the planet's phase curve decreases. That's because in all cases, the planet's dipole is always facing the star. Here's what the planet looks like to us, on the *sky* plane, at a mean anomaly $f = \frac{3\pi}{2} - \omega$ (corresponding to secondary eclipse if the orbit is edge on):

In [ ]:
res = 300
fig, ax = pl.subplots(1, 6, figsize=(16, 4))
x, y = np.meshgrid(np.linspace(-1, 1, res), np.linspace(-1, 1, res))
planet.axis = [1, 0, 0]
for i in range(6):
    I = [planet(theta=90-inclinations[i], x=x[j], y=y[j]) for j in range(res)]
    ax[i].imshow(I, cmap='plasma', origin='lower')
    ax[i].axis('off')
    ax[i].set_title("$i = %d^\circ$" % inclinations[i])

## Rotation in the sky plane

Finally, for the sake of completeness, what about cases where the longitude of ascending node $\Omega \ne 0$? For an edge-on system, this is the angle of rotation of the orbital plane on the sky. Let's see what happens when we specify the same dipole map as before, but vary the longitude of ascending node:

In [ ]:
fig, ax = pl.subplots(2, figsize=(16, 8))
fig.subplots_adjust(hspace=0.5)
planet.inc = 90
planet.ecc = 0
planet[1, 0] = 0.5
Omegas = [0, 30, 45, 60, 90]
for Omega in Omegas:
    planet.Omega = Omega
    system.compute(time)
    ax[0].plot(time, planet.lightcurve, label=Omega);
    ax[1].plot(planet.X, planet.Y, label=Omega)
ax[0].set_xlabel('Time [days]', fontsize=16);
ax[0].set_ylabel('System flux', fontsize=16);
ax[0].legend(title='Omega (deg)', ncol=2);
ax[1].set_ylim(-5, 5);
ax[1].set_xlim(-15, 15);
ax[1].axis('off');

All the light curves are the same, which again is exactly what you'd expect. The longitude of ascending node is not something you can typically probe from a light curve (unless, maybe, with [planet-planet occultations](http://adsabs.harvard.edu/abs/2017ApJ...851...94L))! What this means is that the planet map coefficients are specified as if the planet were seen edge on at secondary eclipse with $\Omega = 0$.

## Summary

Long story short, here's all you need to remember: when you specify the map of a `starry.kepler.Secondary` instance, you are specifying the coefficients of the map you would see if the system were viewed

>1. Edge-on
>2. With the secondary at occultation (i.e., directly behind the primary)
>3. With the orbital plane parallel to the $xy$ axis (that is, $\Omega=0$)

Similarly, the secondary's rotation axis is specified in the same frame.